<a href="https://colab.research.google.com/github/map72ru/data_mining/blob/main/%D0%9A%D0%A0_%D1%80%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install implicit

     |████████████████████████████████| 1.1 MB 2.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.4.8-cp37-cp37m-linux_x86_64.whl size=4606635 sha256=444819a866fad44a3d81150c255fc1b08ba8fad13156bf22f9936c8735dbcb26
  Stored in directory: /root/.cache/pip/wheels/88/e6/34/25e73cccbaf1a961154bb562a5f86123b68fdbf40e306073d6
Successfully built implicit


In [2]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/hw2')

import sys
sys.path.append('/content/hw2/MyDrive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/hw2


In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [4]:
def ap_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(recommended_list, bought_list)

    if sum(flags) == 0:
        return 0

    sum_ = 0
    for i in range(1, k + 1):

        if flags[i - 1] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k

    result = sum_ / min(len(recommended_list), k)

    return result

def map_k(recommended_list, bought_list, k=5):
    assert len(recommended_list) == len(bought_list)

    sum = 0
    for recommended, bought in zip(recommended_list, bought_list):
        sum = sum + ap_k(recommended, bought, k)
        
    return sum/len(recommended_list)

In [5]:
data = pd.read_csv('/content/hw2/MyDrive/data/retail_train.csv')
item_features = pd.read_csv('/content/hw2/MyDrive/data/product.csv')
user_features = pd.read_csv('/content/hw2/MyDrive/data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]
data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [6]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=10000, popularity_level=0.5, max_price=100)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 10001


In [7]:
recommender = MainRecommender(data_train_lvl_1, n_factors=30, iterations=30)

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/10001 [00:00<?, ?it/s]

In [8]:
train_users = data_train_lvl_1['user_id'].unique()
data_val_lvl_1 = data_val_lvl_1[data_val_lvl_1['user_id'].isin(train_users)]

Считаем baseline


In [9]:
result_bsl = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_bsl.columns=['user_id', 'actual']
result_bsl['candidates'] = result_bsl['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=5))
result_bsl.head(2)

,user_id,actual,candidates
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[896666, 1087895, 856942, 877391, 5577022]"
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[1107760, 244960, 932631, 843418, 5567582]"


In [62]:
map5 = 100*map_k(result_bsl['candidates'], result_bsl['actual'], 5)
map5

10.475437780877073

In [11]:
#Посчитаем топ50 для каждого покупателя
result_bsl50 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_bsl50.columns=['user_id', 'actual']
result_bsl50['candidates'] = result_bsl50['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
result_bsl50.head(2)

,user_id,actual,candidates
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[896666, 1087895, 856942, 877391, 5577022, 105..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[1107760, 244960, 932631, 843418, 5567582, 556..."


In [12]:
!pip install lightfm

     |████████████████████████████████| 310 kB 4.0 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705360 sha256=20889e7063c4b873a595f60e3b2db09aa519777f129375a9dc1baa4e7c4392dd
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [13]:
from lightfm import LightFM


In [14]:
users = list(recommender.userid_to_id.keys())
user_features.query('user_id not in @users')

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id


In [16]:
def add_user_features(data_set, user_features):

  cols = data_set[['user_id', 'quantity', 'sales_value', 'day']]
  newdf = cols.copy()
  # количество покупок и сумма всех покупок
  newdf1 = newdf.groupby('user_id').agg({'quantity': 'count', 'sales_value': 'sum'})
  # Средний чек
  newdf1['check']=newdf1['sales_value']/newdf1['quantity']

  newdf2 = newdf.groupby('user_id').agg({'day': 'count', 'quantity': 'sum'})
  # среднее количество покупок за раз
  newdf2['avg_qnt_day'] = newdf2['quantity']/newdf2['day']
  # частота покупок
  newdf2['frq'] = newdf2['day']/len(data_train_lvl_1.day.unique())

  newdf = newdf2.merge(newdf1, how='left', on='user_id')
  cols = newdf[['avg_qnt_day', 'frq', 'sales_value', 'check']]
  newdf = cols.copy()

  if len(user_features.columns.intersection(newdf.columns)) > 0:
    for user_id, row in newdf.iterrows():
      user_features.loc[user_features.index == user_id, 'frq'] = row['frq']
      user_features.loc[user_features.index == user_id, 'user_sales_value'] = row['sales_value']
      user_features.loc[user_features.index == user_id, 'check'] = row['check']
      user_features.loc[user_features.index == user_id, 'chavg_qnt_dayeck'] = row['avg_qnt_day']
    return user_features

  return user_features.merge(newdf, how='left', on='user_id')

In [17]:
user_feat = pd.DataFrame(recommender.user_item_matrix_orig.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)

item_feat = pd.DataFrame(recommender.user_item_matrix_orig.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
user_features_ext = add_user_features(data_train_lvl_1, user_feat)
user_features_ext = add_user_features(data_val_lvl_1, user_features_ext)

user_features_ext.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,avg_qnt_day,frq,sales_value,check,user_sales_value,chavg_qnt_dayeck
user_id,,,,,,,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1.072617,0.224283,2245.23,2.569774,341.78,1.112782
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.099042,0.128162,1244.87,2.503816,190.29,1.105263


In [19]:
def add_items_features(data_set, item_features):
  cols = data_set[['item_id', 'quantity', 'sales_value', 'week_no']]
  newdf = cols.copy()

# Сколько всего продали такого товара и на какую сумму
  newdf1 = newdf.groupby('item_id').agg({'quantity': 'sum', 'sales_value': 'sum'})
  newdf2 = newdf.groupby('item_id').agg({'week_no': 'count', 'quantity':'sum'})
# Среднее количество товаров, продаваемое в неделю
  newdf2['avg_qnt_week'] = newdf2['week_no']/newdf2['quantity']
  newdf2.drop(['quantity', 'week_no'], axis=1, inplace=True);
  newdf2 = newdf1.merge(newdf2, how='left', on='item_id')

  if len(item_features.columns.intersection(newdf2.columns)) > 0:
    for item_id, row in newdf2.iterrows():
        item_features.loc[item_features.index == item_id, 'item_quantity'] = row['quantity']
        item_features.loc[item_features.index == item_id, 'item_sales_value'] = row['sales_value']
        item_features.loc[item_features.index == item_id, 'avg_qnt_week'] = row['avg_qnt_week']
    return item_features

  return item_features.merge(newdf2, how='left', on='item_id')


In [20]:
item_features_ext = add_items_features(data_train_lvl_1, item_feat)
item_features_ext = add_items_features(data_val_lvl_1, item_features_ext)

In [21]:
user_feat_lightfm = pd.get_dummies(user_features_ext, columns=user_features_ext.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_features_ext, columns=item_features_ext.columns.tolist())

In [22]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,income_desc_125-149K,income_desc_15-24K,income_desc_150-174K,income_desc_175-199K,income_desc_200-249K,income_desc_25-34K,income_desc_250K+,income_desc_35-49K,income_desc_50-74K,income_desc_75-99K,income_desc_Under 15K,homeowner_desc_Homeowner,homeowner_desc_Probable Owner,homeowner_desc_Probable Renter,homeowner_desc_Renter,homeowner_desc_Unknown,hh_comp_desc_1 Adult Kids,hh_comp_desc_2 Adults Kids,hh_comp_desc_2 Adults No Kids,hh_comp_desc_Single Female,hh_comp_desc_Single Male,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,...,chavg_qnt_dayeck_1137.5,chavg_qnt_dayeck_1139.9375,chavg_qnt_dayeck_1174.0545454545454,chavg_qnt_dayeck_1178.7906976744187,chavg_qnt_dayeck_1200.4166666666667,chavg_qnt_dayeck_1203.8360655737704,chavg_qnt_dayeck_1228.6363636363637,chavg_qnt_dayeck_1251.3333333333333,chavg_qnt_dayeck_1252.8484848484848,chavg_qnt_dayeck_1286.5142857142857,chavg_qnt_dayeck_1299.9622641509434,chavg_qnt_dayeck_1409.142857142857,chavg_qnt_dayeck_1421.5,chavg_qnt_dayeck_1475.8,chavg_qnt_dayeck_1559.1333333333334,chavg_qnt_dayeck_1662.7142857142858,chavg_qnt_dayeck_1685.3333333333333,chavg_qnt_dayeck_1771.6666666666667,chavg_qnt_dayeck_1777.4782608695652,chavg_qnt_dayeck_2097.6153846153848,chavg_qnt_dayeck_2114.3333333333335,chavg_qnt_dayeck_2175.5,chavg_qnt_dayeck_2398.9736842105262,chavg_qnt_dayeck_2696.5714285714284,chavg_qnt_dayeck_2929.3333333333335,chavg_qnt_dayeck_2933.714285714286,chavg_qnt_dayeck_3037.464285714286,chavg_qnt_dayeck_4349.666666666667,chavg_qnt_dayeck_4547.0,chavg_qnt_dayeck_5407.6,chavg_qnt_dayeck_5650.285714285715,chavg_qnt_dayeck_6738.0,chavg_qnt_dayeck_8410.5,chavg_qnt_dayeck_8699.0,chavg_qnt_dayeck_9095.0,chavg_qnt_dayeck_9222.8,chavg_qnt_dayeck_9444.0,chavg_qnt_dayeck_10068.0,chavg_qnt_dayeck_17326.0,chavg_qnt_dayeck_19603.0
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
item_feat_lightfm.head(2)

,manufacturer_2.0,manufacturer_5.0,manufacturer_8.0,manufacturer_16.0,manufacturer_20.0,manufacturer_26.0,manufacturer_33.0,manufacturer_35.0,manufacturer_36.0,manufacturer_42.0,manufacturer_43.0,manufacturer_46.0,manufacturer_51.0,manufacturer_58.0,manufacturer_61.0,manufacturer_65.0,manufacturer_67.0,manufacturer_69.0,manufacturer_70.0,manufacturer_72.0,manufacturer_76.0,manufacturer_78.0,manufacturer_79.0,manufacturer_80.0,manufacturer_83.0,manufacturer_86.0,manufacturer_87.0,manufacturer_92.0,manufacturer_93.0,manufacturer_97.0,manufacturer_101.0,manufacturer_103.0,manufacturer_105.0,manufacturer_111.0,manufacturer_118.0,manufacturer_121.0,manufacturer_128.0,manufacturer_131.0,manufacturer_135.0,manufacturer_136.0,...,item_sales_value_430.0,item_sales_value_436.68,item_sales_value_436.7800000000001,item_sales_value_447.6100000000003,item_sales_value_449.65000000000003,item_sales_value_449.8200000000001,item_sales_value_452.36000000000024,item_sales_value_455.29000000000025,item_sales_value_458.31000000000034,item_sales_value_459.35000000000025,item_sales_value_479.70000000000016,item_sales_value_481.1900000000006,item_sales_value_485.5200000000005,item_sales_value_490.22000000000014,item_sales_value_492.14000000000027,item_sales_value_515.6899999999996,item_sales_value_517.46,item_sales_value_520.55,item_sales_value_545.1000000000009,item_sales_value_595.5400000000001,item_sales_value_649.270000000001,item_sales_value_649.6799999999997,item_sales_value_670.6699999999967,item_sales_value_677.1400000000001,item_sales_value_678.6200000000017,item_sales_value_707.9199999999998,item_sales_value_719.27,item_sales_value_768.2400000000063,item_sales_value_871.0300000000001,item_sales_value_914.8100000000012,item_sales_value_932.1000000000004,item_sales_value_960.5900000000015,item_sales_value_978.5700000000054,item_sales_value_1105.2900000000013,item_sales_value_1150.640000000001,item_sales_value_1288.49,item_sales_value_1843.1800000000007,item_sales_value_1934.6900000000012,item_sales_value_2002.1200000000003,item_sales_value_2969.3299999999613
item_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
36406,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41332,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
# переведем в формат saprse matrix
sparse_user_item = csr_matrix(recommender.user_item_matrix_orig).tocsr()

In [25]:
# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

In [63]:
model = LightFM(no_components=50,
                loss='bpr',
                learning_rate=0.01, 
                item_alpha=0.1, user_alpha=0.1, 
                random_state=42)

model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4) 

In [64]:
def get_lfm_recommendations(user_id, item_ids, N=5):
    predictions = model.predict(user_ids=int(recommender.userid_to_id[user_id]), item_ids=item_ids,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr()
                            )
    res = pd.DataFrame()
    res['predictions'] = predictions
    res['item_id'] = id_to_itemid.values()
    res1 = res.sort_values('predictions', axis=0, ascending=False)    

    print('.', end='')
    return res1.item_id.values[:N]


In [37]:
def get_items_ids(ids):
  return [recommender.itemid_to_id[x] for x in ids]

In [65]:
temp_result = result_bsl50[:50]

In [66]:
temp_result['lfm_candidates'] = temp_result['user_id'].apply(lambda x:get_lfm_recommendations(x, list(itemid_to_id.values()), N=5))

..................................................

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [68]:
temp_result

,user_id,actual,candidates,lfm_candidates
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[896666, 1087895, 856942, 877391, 5577022, 105...","[1043357, 857348, 842704, 1013308, 832369]"
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[1107760, 244960, 932631, 843418, 5567582, 556...","[1043357, 857348, 842704, 1013308, 832369]"
2,4,"[883932, 970760, 1035676, 1055863, 1097610, 67...","[6391541, 1013868, 979674, 891423, 936470, 109...","[1043357, 857348, 842704, 1013308, 832369]"
3,6,"[1024306, 1102949, 6548453, 835394, 940804, 96...","[962850, 9420159, 923600, 837076, 9858743, 130...","[1043357, 857348, 842704, 1013308, 832369]"
4,7,"[836281, 843306, 845294, 914190, 920456, 93886...","[896666, 998519, 894360, 7147142, 9297027, 557...","[1043357, 857348, 842704, 1013308, 832369]"
5,8,"[868075, 886787, 945611, 1005186, 1008787, 101...","[826578, 926808, 909269, 1069256, 955879, 1280...","[1043357, 857348, 842704, 1013308, 832369]"
6,9,"[883616, 1029743, 1039126, 1051323, 1082772, 1...","[988277, 872146, 12808385, 1028422, 13007284, ...","[1043357, 857348, 842704, 1013308, 832369]"
7,13,"[6544236, 822407, 908317, 1056775, 1066289, 11...","[985178, 9488065, 1075293, 965772, 10342382, 1...","[1043357, 857348, 842704, 1013308, 832369]"
8,14,"[917277, 981760, 878234, 925514, 986394, 10220...","[902377, 822161, 1026439, 870249, 874563, 8943...","[1043357, 857348, 842704, 1013308, 832369]"
9,15,"[996016, 1014509, 1044404, 1087353, 976199, 10...","[1037850, 823576, 986293, 1052975, 955879, 107...","[1043357, 857348, 842704, 1013308, 832369]"


In [67]:
map5 = 100*map_k(temp_result['lfm_candidates'], temp_result['actual'], 5)
map5

0.0

Дальше просто всякие проверки

In [93]:
predictions = model.predict(user_ids=1, item_ids=list(itemid_to_id.values()),
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr()
                            )
predictions

array([-4.4926467, -4.063368 , -4.0445385, ..., -4.0783176, -4.0266995,
       -4.536518 ], dtype=float32)

In [94]:
predictions1 = model.predict(user_ids=2, item_ids=list(itemid_to_id.values()),
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr()
                            )
predictions1

array([-3.5727592, -3.143416 , -3.1246645, ..., -3.1584384, -3.1068115,
       -3.616636 ], dtype=float32)

In [83]:
max(predictions1)

-0.92018294

In [95]:
res = pd.DataFrame()
res['predictions'] = predictions
res['item_id'] = id_to_itemid.values()
res1 = res.sort_values('predictions', axis=0, ascending=False)
res1

,predictions,item_id
3907,-0.920183,1043357
684,-3.200942,857348
419,-3.221717,842704
3417,-3.251674,1013308
236,-3.284358,832369
...,...,...
1748,-4.669130,917384
6088,-4.670653,6533368
8004,-4.675965,14025700
4160,-4.679481,1059067


In [96]:
res = pd.DataFrame()
res['predictions'] = predictions1
res['item_id'] = id_to_itemid.values()
res1 = res.sort_values('predictions', axis=0, ascending=False) 
res1  

,predictions,item_id
3907,-0.000298,1043357
684,-2.281042,857348
419,-2.301831,842704
3417,-2.331779,1013308
236,-2.364475,832369
...,...,...
1748,-3.749215,917384
6088,-3.750741,6533368
8004,-3.756073,14025700
4160,-3.759598,1059067


In [70]:
get_lfm_recommendations(1, list(itemid_to_id.values()))

.

array([1043357,  857348,  842704, 1013308,  832369])

In [71]:
get_lfm_recommendations(2, list(itemid_to_id.values()))

.

array([1043357,  857348,  842704, 1013308,  832369])

0.19974247545469187